In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Next_Word_Predictor\\research'

In [3]:
os.chdir("../")

In [4]:

%pwd

'c:\\DataScience\\Projects\\Next_Word_Predictor'

In [5]:
import joblib
import numpy as np
import pandas as pd
from NWPproject.constants import *
from NWPproject.logging import logger
from NWPproject.utils.common import read_yaml_file
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class PredictionConfig:
    input_length:int
    

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        params_filepath = PARAMS_FILE_PATH):
        # schema_filepath = SCHEMA_FILE_PATH):

        self.params = read_yaml_file(params_filepath)
        # self.schema = read_yaml_file(schema_filepath)

    def get_prediction_config(self) -> PredictionConfig:
        # config = self.config.prediction
        input_length_params = self.params.Embedding


        prediction_config = PredictionConfig(
            input_length = input_length_params.input_length  
        )

        return prediction_config

In [8]:
class PredictionPipeline:
    def __init__(self,config:PredictionConfig):
        self.config = config
        self.model = joblib.load(TRAINED_MODEL_PATH)
        self.token = joblib.load(TOKEN_PATH)

    def predict(self, data):        
        res=[]

        print(data)

        for i in range(len(data.split(" "))):
            token_text=self.token.texts_to_sequences([data])[0]

            print(token_text)

            padded_token_text=pad_sequences([token_text],maxlen=self.config.input_length,padding="pre")

            idx=np.argmax(self.model.predict(padded_token_text))

            for word,index in self.token.word_index.items():
                if index==idx:
                    data=data+" "+ word
                    res.append( data)

        # prediction = int(self.model.predict(data))
        logger.info(f"Predicted Word: {res}")
        return res

In [9]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction= PredictionPipeline(config=prediction_config)
    PW=prediction.predict("big story")
except Exception as e:
    raise e

[2024-07-03 22:14:50,232: INFO:common: yaml file: params.yaml loaded successfully]
big story
[206, 214]
1/1 [==============================] - 0s 360ms/step
[206, 214, 6]
1/1 [==============================] - 0s 21ms/step
[2024-07-03 22:14:51,041: INFO:2500965802: Predicted Word: ['big story and', 'big story and then']]


In [10]:
PW

['big story and', 'big story and then']